### 1. Instalação de Pacotes

In [2]:
# Executar no terminal
!pip install lightgbm pandas polars pyarrow numpy scikit-learn joblib

### 2. Importações

In [3]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

### 3. Carregamento dos Dados

In [4]:
#Caminhos dos arquivos
product_catalog_path = r"C:\Users\Pedro\Documents\hackathon_2025_templates\part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet"
transactions_path = r"C:\Users\Pedro\Downloads\hackathon_2025_templates\part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet"
pdv_catalog_path = r"C:\Users\Pedro\Downloads\hackathon_2025_templates\part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet"

df_produtos = pd.read_parquet(product_catalog_path)
df_transacoes = pd.read_parquet(transactions_path)
df_pdvs = pd.read_parquet(pdv_catalog_path)

print("Catálogo de produtos:", df_produtos.columns.tolist())
print("Histórico de vendas:", df_transacoes.columns.tolist())
print("Catálogo de PDVs:", df_pdvs.columns.tolist())
print("Qtd linhas produtos:", df_produtos.shape[0])
print("Qtd linhas vendas:", df_transacoes.shape[0])
print("Qtd linhas PDVs:", df_pdvs.shape[0])

Catálogo de produtos: ['produto', 'categoria', 'descricao', 'tipos', 'label', 'subcategoria', 'marca', 'fabricante']
Histórico de vendas: ['internal_store_id', 'internal_product_id', 'distributor_id', 'transaction_date', 'reference_date', 'quantity', 'gross_value', 'net_value', 'gross_profit', 'discount', 'taxes']
Catálogo de PDVs: ['pdv', 'premise', 'categoria_pdv', 'zipcode']
Qtd linhas produtos: 7092
Qtd linhas vendas: 6560698
Qtd linhas PDVs: 14419


### 4. Pré-processamento das Transações

In [5]:
df_transacoes['transaction_date'] = pd.to_datetime(df_transacoes['transaction_date'])
df_transacoes['semana'] = df_transacoes['transaction_date'].dt.isocalendar().week
df_transacoes['ano'] = df_transacoes['transaction_date'].dt.year
df_2022 = df_transacoes[df_transacoes['ano'] == 2022]

### 5. Agregação de Vendas Semanais

In [6]:
df_agregado = df_2022.groupby(['semana', 'internal_store_id', 'internal_product_id'])['quantity'].sum().reset_index()
df_agregado.columns = ['semana', 'pdv', 'produto', 'quantidade']

### 6. Feature: Média Móvel de 4 Semanas

In [7]:
df_agregado = df_agregado.sort_values(by=['pdv', 'produto', 'semana'])
df_agregado['media_movel_4s'] = (
    df_agregado.groupby(['pdv', 'produto'])['quantidade']
    .transform(lambda x: x.rolling(window=4, min_periods=1).mean())
)

### 7. Enriquecimento com Catálogos

In [8]:
df_agregado = df_agregado.merge(df_produtos, on='produto', how='left')
df_agregado = df_agregado.merge(df_pdvs, on='pdv', how='left')

### 8. Codificação de Variáveis Categóricas

In [9]:
le_categoria = LabelEncoder()
le_subcategoria = LabelEncoder()
le_marca = LabelEncoder()
le_categoria_pdv = LabelEncoder()

df_agregado['categoria'] = le_categoria.fit_transform(df_agregado['categoria'].astype(str))
df_agregado['subcategoria'] = le_subcategoria.fit_transform(df_agregado['subcategoria'].astype(str))
df_agregado['marca'] = le_marca.fit_transform(df_agregado['marca'].astype(str))
df_agregado['categoria_pdv'] = le_categoria_pdv.fit_transform(df_agregado['categoria_pdv'].astype(str))

### 9. Separação de Features e Target

In [10]:
#Remoção de outliers extremos
q99 = df_agregado['quantidade'].quantile(0.99)
df_agregado = df_agregado[df_agregado['quantidade'] <= q99]

#Aplicação de log com proteção
df_agregado['quantidade'] = df_agregado['quantidade'].clip(lower=1)
target = np.log1p(df_agregado['quantidade'])

features = df_agregado[['semana', 'media_movel_4s', 'categoria', 'subcategoria', 'marca', 'categoria_pdv']]
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)

### 10. Treinamento do Modelo

In [11]:
modelo_lgb = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
modelo_lgb.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020830 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 662
[LightGBM] [Info] Number of data points in the train set: 4950289, number of used features: 6
[LightGBM] [Info] Start training from score 1.283927


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


### 11. Avaliação do Modelo

In [12]:
#Previsão e reversão do log
y_pred_log = modelo_lgb.predict(X_test)
y_test_log = y_test.copy()

limite_log = 20
y_pred_log = np.clip(y_pred_log, a_min=None, a_max=limite_log)
y_test_log = np.clip(y_test_log, a_min=None, a_max=limite_log)

y_pred = np.expm1(y_pred_log)
y_test_original = np.expm1(y_test_log)

#Filtragem de valores válidos
mask_valid = (
    ~np.isnan(y_test_original) &
    ~np.isnan(y_pred) &
    ~np.isinf(y_test_original) &
    ~np.isinf(y_pred)
)

y_test_valid = y_test_original[mask_valid]
y_pred_valid = y_pred[mask_valid]

#Métricas
mae = mean_absolute_error(y_test_valid, y_pred_valid)
rmse = np.sqrt(mean_squared_error(y_test_valid, y_pred_valid))
mask = y_test_valid != 0
mape = np.mean(np.abs((y_test_valid[mask] - y_pred_valid[mask]) / y_test_valid[mask])) * 100
acertos = np.abs(y_test_valid - y_pred_valid) <= 0.1 * y_test_valid
porcentagem_acerto = acertos.mean() * 100

print(f"\n📈 Avaliação do modelo LightGBM:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")
print(f"Acerto (±10%): {porcentagem_acerto:.2f}%")


📈 Avaliação do modelo LightGBM:
MAE: 1.08
RMSE: 3.58
MAPE: 26.63%
Acerto (±10%): 48.13%


### 12. Diagnóstico Detalhado

In [13]:
df_avaliacao = pd.DataFrame({'real': y_test_valid, 'previsto': y_pred_valid})
df_avaliacao['faixa'] = pd.cut(df_avaliacao['real'], bins=[-1, 0, 5, 20, 100, np.inf], labels=['zero', 'baixo', 'médio', 'alto', 'muito alto'])

print("\nAcerto por faixa de volume:")
print(df_avaliacao.groupby('faixa').apply(lambda x: np.mean(np.abs(x['real'] - x['previsto']) <= 0.1 * x['real'])))

print("\nDistribuição de vendas reais:")
print(df_avaliacao['real'].describe())

print("\nDistribuição de previsões:")
print(df_avaliacao['previsto'].describe())

print("\nErros absolutos:")
print(np.abs(df_avaliacao['real'] - df_avaliacao['previsto']).describe())

print("\nMAPE por faixa:")
def mape_por_faixa(grupo):
    mask = grupo['real'] != 0
    return np.mean(np.abs((grupo['real'][mask] - grupo['previsto'][mask]) / grupo['real'][mask])) * 100
print(df_avaliacao.groupby('faixa').apply(mape_por_faixa))


Acerto por faixa de volume:
faixa
zero               NaN
baixo         0.495318
médio         0.392187
alto          0.518740
muito alto    0.691049
dtype: float64

Distribuição de vendas reais:
count    1.237573e+06
mean     4.874890e+00
std      1.054153e+01
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      4.000000e+00
max      1.200000e+02
Name: real, dtype: float64

Distribuição de previsões:
count    1.237573e+06
mean     4.565648e+00
std      9.379942e+00
min      8.952368e-01
25%      1.007442e+00
50%      1.872083e+00
75%      3.896889e+00
max      1.361501e+02
Name: previsto, dtype: float64

Erros absolutos:
count    1.237573e+06
mean     1.081353e+00
std      3.416340e+00
min      7.982680e-07
25%      7.101274e-03
50%      2.825916e-01
75%      8.973664e-01
max      1.113642e+02
dtype: float64

C:\Users\Pedro\AppData\Local\Temp\ipykernel_524\3237088758.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df_avaliacao.groupby('faixa').apply(lambda x: np.mean(np.abs(x['real'] - x['previsto']) <= 0.1 * x['real'])))
C:\Users\Pedro\AppData\Local\Temp\ipykernel_524\3237088758.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  print(df_avaliacao.groupby('faixa').apply(lambda x: np.mean(np.abs(x['real'] - x['previsto']) <= 0.1 * x['real'])))




MAPE por faixa:
faixa
zero                NaN
baixo         27.753295
médio         22.567837
alto          19.623219
muito alto    16.747753
dtype: float64


C:\Users\Pedro\AppData\Local\Temp\ipykernel_524\3237088758.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df_avaliacao.groupby('faixa').apply(mape_por_faixa))
C:\Users\Pedro\AppData\Local\Temp\ipykernel_524\3237088758.py:20: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  print(df_avaliacao.groupby('faixa').apply(mape_por_faixa))


### 13. Previsão para Janeiro/2023

In [14]:
#Pares reais de PDV/SKU com histórico
pares_reais = df_agregado[['pdv', 'produto']].drop_duplicates()

#Semanas de janeiro
semanas_jan = [1, 2, 3, 4, 5]

#Gerar grade combinando semanas com pares reais
df_janeiro = pd.DataFrame([
    {'semana': semana, 'pdv': row['pdv'], 'produto': row['produto']}
    for semana in semanas_jan
    for _, row in pares_reais.iterrows()
])

#Enriquecer com atributos dos catálogos
df_janeiro = df_janeiro.merge(df_produtos, on='produto', how='left')
df_janeiro = df_janeiro.merge(df_pdvs, on='pdv', how='left')

#Codificação categórica (usando os mesmos LabelEncoders do treino)
df_janeiro['categoria'] = le_categoria.transform(df_janeiro['categoria'].astype(str))
df_janeiro['subcategoria'] = le_subcategoria.transform(df_janeiro['subcategoria'].astype(str))
df_janeiro['marca'] = le_marca.transform(df_janeiro['marca'].astype(str))
df_janeiro['categoria_pdv'] = le_categoria_pdv.transform(df_janeiro['categoria_pdv'].astype(str))

#Média móvel como proxy de histórico
media_movel = df_agregado.groupby(['pdv', 'produto'])['media_movel_4s'].mean().reset_index()
df_janeiro = df_janeiro.merge(media_movel, on=['pdv', 'produto'], how='left')
df_janeiro['media_movel_4s'] = df_janeiro['media_movel_4s'].fillna(0)

#Previsão com modelo treinado
X_jan = df_janeiro[['semana', 'media_movel_4s', 'categoria', 'subcategoria', 'marca', 'categoria_pdv']]
df_janeiro['quantidade'] = modelo_lgb.predict(X_jan).round().astype(int)

#Seleção final de colunas
df_entrega = df_janeiro[['semana', 'pdv', 'produto', 'quantidade']]

#Exportar CSV
df_entrega.to_csv("previsao_janeiro_2023.csv", index=False, encoding='utf-8')
print("\nArquivo 'previsao_janeiro_2023.csv' salvo com sucesso!")



Arquivo 'previsao_janeiro_2023.csv' salvo com sucesso!


In [15]:
print(df_entrega.head(10))
print(df_entrega.dtypes)

   semana                  pdv              produto  quantidade
0       1  1000237487041964405  1837429607327399565           1
1       1  1000237487041964405  4038588102284338370           1
2       1  1000237487041964405  5429216175252037173           1
3       1  1000237487041964405   596381974901127871           1
4       1  1000237487041964405  7270233133454638680           1
5       1  1000237487041964405  7370044109082767116           1
6       1  1000237487041964405  7405304019373961901           1
7       1  1000237487041964405   777251454728290683           1
8       1  1000237487041964405  8313805606242965556           1
9       1  1000275275922029725  1735457469340543861           1
semana         int64
pdv           object
produto       object
quantidade     int64
dtype: object
